## SRJ

    Active space finder

    1. In this, I included active space finder code. I changed the rhf.mo_coeff to active_space.mo_coeff
        I got the energy same as A-code. Error: 2.15e-6

In [1]:
import basis_set_exchange as bse
basis = bse.get_basis('ANO-RCC-MB', elements=['Fe','S'], fmt='nwchem')

In [8]:
import pyscf
pyscf.__config__.B3LYP_WITH_VWN5 = False
from pathlib import Path

# The Mole class is used to define molecular information in PySCF.
from pyscf.gto import Mole
from pyscf import scf

# logger contains definitions of verbosity levels for PySCF.
from pyscf.lib import logger

# Functionality for (state-averaged) CASSCF.
from pyscf.mcscf import CASSCF, CASCI, state_average_mix
from pyscf.fci.direct_spin1 import FCISolver
from pyscf.fci.addons import fix_spin

# Wrapper functions to perform selection for variable and fixed active space sizes
from asf.wrapper import find_from_mol, find_from_scf, sized_space_from_mol, sized_space_from_scf
from asf.scf import stable_scf
# Various utility functions...
from asf.utility import compare_active_spaces, show_mos_grid, pictures_Jmol

from asf import ASFDMRG
from asf.visualization import draw_pair_information
from asf.preselection import MP2NatorbPreselection, MP2PairinfoPreselection
from asf.scf import stable_scf
from asf.utility import pictures_Jmol
mol = Mole()
mol.atom = '''
Fe   0.0   0.0   0.0
S   0.0   0.0   1.826
'''
mol.basis = basis
mol.charge = 0
mol.spin = 4
# Set mol.verbose = logger.INFO to enable printing of SCF iterations and further output.
mol.verbose = logger.NOTE
mol.build()
mf = scf.ROHF(mol) # or scf.RHF as appropriate
#mf.chkfile = 'FeS.chk'
mf.kernel()

print("Atom list:", mol.atom)
#print("Basis used:", mol.basis)
print("Charge:", mol.charge)
print("Spin:", mol.spin)
print("Number of electrons:", mol.nelec)
n_orb = mf.mo_coeff.shape[0]
print("Number of molecular orbitals:", n_orb)

print('Going to select Active space')
print('Orbital count', mol.nao)

for i, (e, occ) in enumerate(zip(mf.mo_energy, mf.mo_occ)):
    print(f"MO #{i:2d}: Energy = {e:10.6f} Ha, Occupancy = {occ}")


#This find_from_mol is taken from asf
active_space = find_from_mol(mol, max_norb=4)

print('Act space',active_space)

#This CASCI is taken from pyscf
mc = CASCI(mol, ncas=active_space.norb, nelecas=active_space.nel)

print('MO list',active_space.mo_list)
#print('act space Mo coeff', active_space.mo_coeff)

#Original 
#mo_guess = mc.sort_mo(active_space.mo_list, active_space.mo_coeff, base=0)
#Sri line
mo_guess = mc.sort_mo(active_space.mo_list, active_space.mo_coeff, base=0)

_ = mc.kernel(mo_coeff=mo_guess, verbose=3) 

print('CASCI total energy - Sri', mc.e_tot)
#print('Active space energy- Sri', mc.e_cas)

converged SCF energy = -1655.34968383623
Atom list: 
Fe   0.0   0.0   0.0
S   0.0   0.0   1.826

Charge: 0
Spin: 4
Number of electrons: (23, 19)
Number of molecular orbitals: 27
Going to select Active space
Orbital count 27
MO # 0: Energy = -259.180107 Ha, Occupancy = 2.0
MO # 1: Energy = -91.641815 Ha, Occupancy = 2.0
MO # 2: Energy = -31.779132 Ha, Occupancy = 2.0
MO # 3: Energy = -27.441840 Ha, Occupancy = 2.0
MO # 4: Energy = -27.441840 Ha, Occupancy = 2.0
MO # 5: Energy = -27.420625 Ha, Occupancy = 2.0
MO # 6: Energy =  -8.859585 Ha, Occupancy = 2.0
MO # 7: Energy =  -6.547802 Ha, Occupancy = 2.0
MO # 8: Energy =  -6.541798 Ha, Occupancy = 2.0
MO # 9: Energy =  -6.541798 Ha, Occupancy = 2.0
MO #10: Energy =  -4.269029 Ha, Occupancy = 2.0
MO #11: Energy =  -2.875990 Ha, Occupancy = 2.0
MO #12: Energy =  -2.866330 Ha, Occupancy = 2.0
MO #13: Energy =  -2.866330 Ha, Occupancy = 2.0
MO #14: Energy =  -0.897661 Ha, Occupancy = 2.0
MO #15: Energy =  -0.646508 Ha, Occupancy = 2.0
MO #16:

In [3]:
import basis_set_exchange as bse
print('Check if I am using ANO-RCC-MB basis set')
print('Using newton method ')
basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')

import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

#print('Using newton method')
# Your original setup (unchanged) 
mol_pyscf = gto.M(
    atom = 'Fe 0 0 0; S 0 0 1.826',
    basis = basis_dict,
    symmetry = False, 
    spin = 4)
rhf = scf.ROHF(mol_pyscf)
print('Molecular point group:', mol_pyscf.topgroup)
rhf.max_cycle = 5000
#rhf.init_guess = "hcore"
#rhf.level_shift = 0.0    #This command is changing the energy 
#rhf.damp = 0.2
#rhf.diis_start_cycle = 5

#rhf = rhf.newton() 
e = rhf.kernel()



for i, (e, occ) in enumerate(zip(rhf.mo_energy, rhf.mo_occ)):
    print(f"MO #{i:2d}: Energy = {e:10.6f} Ha, Occupancy = {occ}")

import pennylane as qml 
from pennylane import qchem

#Atomic integrals - calculations done and understood 
one_ao = mol_pyscf.intor_symmetric("int1e_kin") + mol_pyscf.intor_symmetric("int1e_nuc")   #intor_symmetric : One -electron integral generator. Integrals are assumed here to be hermitian (https://pyscf.org/_modules/pyscf/gto/mole.html#MoleBase.intor_symmetric)
two_ao = mol_pyscf.intor("int2e_sph")  #eri electron repulsion integrals - Tried using int2e, still same results

#Molecular integrals 
one_mo = np.einsum("pi,pq,qj->ij", active_space.mo_coeff, one_ao, active_space.mo_coeff, optimize=True)
two_mo = ao2mo.incore.full(two_ao, active_space.mo_coeff)




core_constant = np.array([rhf.energy_nuc()])


# convert the two-electron integral tensor to the physicists’ notation
two_mo = np.swapaxes(two_mo, 1, 3)
#print('Shape after swapping axes', two_mo.shape)

#Core and active are received from the previous code
#core = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,17,18]
active_list = active_space.mo_list


active = active_space.mo_list
print('active orbitals ', active)
core = list(range(min(active_list)))
print('core orb', core)
total_orb = len(core) + len(active)
print('Total orbitals are',total_orb)



# define the active space and recompute the integrals
def pyscf_integral(core, active, one_mo, two_mo, core_constant):
    if core and active:
        for i in core:
            #print('The value of i going in ', i)
            core_constant = core_constant + 2 * one_mo[i][i]
            #print('core cosntant', core_constant)
            for j in core:
                #print('Value of i and j ', i,j)
                core_constant = core_constant + 2 * two_mo[i][j][j][i] - two_mo[i][j][i][j]
        print('The final value of core constant', core_constant)

        for p in active:
            for q in active:
                for i in core:
                    #print(f'The value of p and q', {p}, {q})
                    one_mo[p, q] = one_mo[p, q] + (2 * two_mo[i][p][q][i] - two_mo[i][p][i][q])

        one_mo = one_mo[qml.math.ix_(active, active)]
        two_mo = two_mo[qml.math.ix_(active, active, active, active)]
        print('Within pyscf integral one_mo shape', one_mo.shape)
        print('Within pyscf integral one_mo shape', one_mo.shape)
    return core_constant, one_mo, two_mo

core_constant, one_mo, two_mo = pyscf_integral(core, active, one_mo,two_mo, core_constant)


H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo, cutoff=1e-20)

H = qml.jordan_wigner(H_fermionic)

print(H)

Check if I am using ANO-RCC-MB basis set
Using newton method 
Molecular point group: C1
converged SCF energy = -1655.34968383623
MO # 0: Energy = -259.180107 Ha, Occupancy = 2.0
MO # 1: Energy = -91.641815 Ha, Occupancy = 2.0
MO # 2: Energy = -31.779132 Ha, Occupancy = 2.0
MO # 3: Energy = -27.441840 Ha, Occupancy = 2.0
MO # 4: Energy = -27.441840 Ha, Occupancy = 2.0
MO # 5: Energy = -27.420625 Ha, Occupancy = 2.0
MO # 6: Energy =  -8.859585 Ha, Occupancy = 2.0
MO # 7: Energy =  -6.547802 Ha, Occupancy = 2.0
MO # 8: Energy =  -6.541798 Ha, Occupancy = 2.0
MO # 9: Energy =  -6.541798 Ha, Occupancy = 2.0
MO #10: Energy =  -4.269029 Ha, Occupancy = 2.0
MO #11: Energy =  -2.875990 Ha, Occupancy = 2.0
MO #12: Energy =  -2.866330 Ha, Occupancy = 2.0
MO #13: Energy =  -2.866330 Ha, Occupancy = 2.0
MO #14: Energy =  -0.897661 Ha, Occupancy = 2.0
MO #15: Energy =  -0.646508 Ha, Occupancy = 2.0
MO #16: Energy =  -0.390507 Ha, Occupancy = 2.0
MO #17: Energy =  -0.390507 Ha, Occupancy = 2.0
MO #18

In [4]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []
gs_energy = []
operator_check = []  # To store the highest gradient excitation operators
theta = np.pi/4
print('Theta is', theta)

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity




#symbols = ["H","H","H","H"]
electrons = 4
qubits = 8
#orbitals = 8
#r_bohr = bond_length *1.8897259886 
##r_bohr = bond_length * 1.0  # Convert bond length to Bohr radius 
#coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
#H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf")
#hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Preparation of the Hartree-Fock state in the form of 2^8 states
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def hf_stateprep(wires):
        target_state = np.zeros(2**qubits)
        target_state[240] = 1.0 # Every molecule change, you need to change this index
        qml.StatePrep(target_state, wires=range(qubits))
        return qml.state()

    hf_state = hf_stateprep(wires=qubits)
    #print('HF state is', hf_state)

    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.StatePrep(hf_state, wires=range(qubits))
        return qml.expval(H)  
    
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)
    #Applyign the givens rotation to the HF state
    @qml.qnode(dev)
    def new_state(hf_state, ash_excitation, qubits):
        qml.StatePrep(hf_state, wires=range(qubits))
        for i in range(len(ash_excitation)):
            if len(ash_excitation[i]) == 4:
                qml.DoubleExcitation(theta, wires=ash_excitation[i])
            elif len(ash_excitation[i]) == 2:
                qml.SingleExcitation(theta, wires=ash_excitation[i])
        return qml.state()
    
    #Measurement of new state
    dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev_meas)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()
    
    @qml.qnode(dev)
    def ind_state(ash_excitation):
        qml.StatePrep(hf_state, wires=range(qubits))
        #print('Individual excitation is', ash_excitation)
        if len(ash_excitation) == 4 :
            qml.DoubleExcitation(theta, wires=ash_excitation)
        elif len(ash_excitation) == 2:
            qml.SingleExcitation(theta, wires=ash_excitation)
        return qml.state()


    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, qubits)

    
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('The operator pool length is', len(operator_pool))
    states = [hf_state]
    max_operator = None

    for j in range(1, adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        #max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        counts1 = measure(k)  #Measure the current state
        #print('The measurement of new state is', counts1)
        #print(f'The operator check for this iteration {j} is', operator_check)
        for i in operator_pool:
            if i not in operator_check:  # Check if the operator is already used
                #print('The current excitation operator is', i) #Current excitation operator - fermionic one
                #print('The value of  operator check is', operator_check)  #Max excitation operator
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
                #print(f'The expectation value of {i} is', current_value)

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

        #print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value
        operator_check.append(max_operator)
        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations1 = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations1)
        ash_excitation.append(excitations1)
        #print('ash_excitation is', ash_excitation)
        #print('The length of ash_excitation before generating matrix is', len(ash_excitation))

        
        #Now apply the givens rotation for it. 
        ostate = new_state(hf_state, ash_excitation, qubits)
        #print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation, qubits))
        #Measuring the state
        counts = measure(ostate)
        #print('State count after Generating function is', counts)
        if j >= 2:
            states.append(ind_state(ash_excitation[-1]))  # Append the individual excitation state
        else:
            print('Skipping individual excitation state for j < 2')
        states.append(ostate)
        #print(f'After adding the states the adapt iteration is {j} and no of states',len(states))
        M = np.zeros((len(states), len(states)), dtype=complex)  # Initialize H with zeros
        S = np.zeros((len(states), len(states)), dtype=complex)  # Initialize S matrix with zeros
        Ham_matrix = qml.matrix(H, wire_order=range(qubits))  # Get the Hamiltonian matrix
        #print('Shape of Hamiltonian matrix is', Ham_matrix.shape)
        # Now we need to fill the H and S matrices
        for i in range(len(states)):
            for j in range(len(states)):
                #print('The value of i and j is', i, j)
                left_op = states[i].T.conj()
                right_op = states[j]
                M[i,j] = left_op.dot(Ham_matrix.dot(right_op)).real
                S[i,j] = left_op.dot(right_op).real
     
                
                if i != j:
                    M[j,i] = M[i,j].conj()
                    S[j,i] = S[i,j].conj()
        n = S.shape[0]
        epsilon = 1e-10
        S_reg = S + epsilon * np.eye(n)
        #print(f'S matrix with {i} and {j} is', S)
        #print('Hamiltonian Matrix with', i, j, 'is', M)
        #print('The shape of S matrix is', S.shape)  
        eig,evec=scipy.linalg.eigh(M,S_reg)
        #print('Eigenvalues are', eig)
        gs_energy.append(eig[0])
        print('Ground state energy is', gs_energy) 
        print('\n')
          
    return ash_excitation, states,eig,gs_energy, Ham_matrix,hf_state


Theta is 0.7853981633974483


In [5]:
ash_excitation, states,eig,gs_energy,Ham_matrix, hf_state = adaptvqe(adapt_it=15, e_th=1e-12)

HF state is -1655.1571584004187
The operator pool length is 26
The adapt iteration now is 1


/Users/s.poyyapakkam/miniconda3/envs/FeS/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


Highest gradient excitation is [2, 3, 6, 7]
Skipping individual excitation state for j < 2
Ground state energy is [np.float64(-1655.1667641328545)]


The adapt iteration now is 2
Highest gradient excitation is [0, 1, 4, 5]
Ground state energy is [np.float64(-1655.1667641328545), np.float64(-1655.1785905123795)]


The adapt iteration now is 3
Highest gradient excitation is [0, 1, 6, 7]
Ground state energy is [np.float64(-1655.1667641328545), np.float64(-1655.1785905123795), np.float64(-1655.1818224343049)]


The adapt iteration now is 4
Highest gradient excitation is [2, 3, 4, 5]
Ground state energy is [np.float64(-1655.1667641328545), np.float64(-1655.1785905123795), np.float64(-1655.1818224343049), np.float64(-1655.1818364056778)]


The adapt iteration now is 5
Highest gradient excitation is [1, 2, 5, 6]
Ground state energy is [np.float64(-1655.1667641328545), np.float64(-1655.1785905123795), np.float64(-1655.1818224343049), np.float64(-1655.1818364056778), np.float64(-1655.2358131782

In [6]:
gs_energy[-1]

np.float64(-1655.3335522113882)

In [10]:
FCI = mc.e_tot
print('FCI using active space', FCI)
gs_energy[-1] - FCI

FCI using active space -1655.3335551833436


np.float64(2.9719553822360467e-06)